<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/%EA%B3%BC%EC%A0%95/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8F%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%86%8C%EB%B6%84%EB%A5%98_%ED%95%99%EC%8A%B5%EA%B2%B0%EA%B3%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 2.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595745 sha256=02b68fc91913c392484f6ed907cc693feb9554bb74b4cffbc67f7df8b9015c7d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 2.1 MB/s 
     |████████████████████████████████| 769 kB 2.0 MB/s 
     |████████████████████████████████| 3.0 MB 68.1 MB/s 
     |████████████████████████████████| 895 kB 78.0 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-8dbbpjwf
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본   산업분류자동화	'Colab Notebooks'  'My Drive'


In [48]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")
ts=pd.read_csv('/mydrive/산업분류자동화/답안 작성용 파일.csv', encoding = "euc-kr")

In [49]:
df=df[df['digit_1']=='F']
ts=ts[ts['digit_1']=='F']

In [50]:
df=df.reset_index(drop=True)

## 데이터 전처리

In [51]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000010,F,42,423,영업장에서,고객의뢰를 받아,내부전기공사
1,id_0000052,F,42,422,사업장에서,의뢰.요청받아,상수도배관공사
2,id_0000197,F,41,411,현장에서,고개의 요청에따라,주택건설
3,id_0000222,F,42,422,건축물내,"기계,장비설치공사",공조기 설치
4,id_0000227,F,42,424,건물내,고객요구에 따라,인테리어공사
5,id_0000257,F,42,421,전문직 건설,수중공사 전문직수행,수중펌프
6,id_0000282,F,42,422,현장에서,고객의뢰에따라,전기소방시공
7,id_0000325,F,42,424,건물내,고객요구에 따라,인테리어공사
8,id_0000346,F,42,421,사업장에서,고객의 요청으로,종합건설서비스
9,id_0000541,F,42,424,사업장에서,계약에의해,실내장식


In [52]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000010,F,42,423,영업장에서 고객의뢰를 받아 내부전기공사
1,id_0000052,F,42,422,사업장에서 의뢰.요청받아 상수도배관공사
2,id_0000197,F,41,411,현장에서 고개의 요청에따라 주택건설
3,id_0000222,F,42,422,"건축물내 기계,장비설치공사 공조기 설치"
4,id_0000227,F,42,424,건물내 고객요구에 따라 인테리어공사
...,...,...,...,...,...
35045,id_0999849,F,42,424,토탈인테리어및 유리바닥재도소매
35046,id_0999875,F,41,412,건설현장에서 고객의 의뢰를 받아 토목공사
35047,id_0999989,F,41,412,현장에서 건설업 토목시설물
35048,id_0999993,F,41,412,고객의사업장에서 고객요구의따라 조경건설공사


In [53]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [54]:
df.shape

(35050, 5)

In [55]:
df['digit_2'].value_counts()

42    28105
41     6945
Name: digit_2, dtype: int64

In [56]:
df['digit_3'].value_counts().sort_index()

411     3582
412     3363
421     4724
422     5744
423     4360
424    11152
425      559
426     1566
Name: digit_3, dtype: int64



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [57]:
df.loc[(df['digit_3'] == 411), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 412), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 421), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 422), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 423), 'digit_3'] = 4
df.loc[(df['digit_3'] == 424), 'digit_3'] = 5
df.loc[(df['digit_3'] == 425), 'digit_3'] = 6
df.loc[(df['digit_3'] == 426), 'digit_3'] = 7

In [58]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['영업장에서 고객의뢰를 받아 내부전기공사', '4'], ['사업장에서 의뢰.요청받아 상수도배관공사', '3'], ['현장에서 고개의 요청에따라 주택건설', '0'], ['건축물내 기계,장비설치공사 공조기 설치', '3'], ['건물내 고객요구에 따라 인테리어공사', '5']]


## Train data & test data

In [59]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [60]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [61]:
# Setting parameters
max_len = 64
batch_size = 64 #32,16,8로 감소시키기
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [62]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [63]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [64]:
data_train[0]

(array([   2, 2600, 6903,  994, 7095, 6999, 7431, 7078, 4257, 7181, 7012,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(12, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 7)

In [65]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [66]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [67]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [68]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/493 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.1483936309814453 train acc 0.0625
epoch 1 batch id 201 loss 0.40991640090942383 train acc 0.6068097014925373
epoch 1 batch id 401 loss 0.35027459263801575 train acc 0.7501168952618454
epoch 1 train acc 0.7773438890075086


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.9110157699443413


  0%|          | 0/493 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.09900707006454468 train acc 0.984375
epoch 5 batch id 201 loss 0.08246352523565292 train acc 0.9520366915422885
epoch 5 batch id 401 loss 0.12788261473178864 train acc 0.9541380922693267
epoch 5 train acc 0.953850063165012


  0%|          | 0/55 [00:00<?, ?it/s]

epoch 5 test acc 0.9127203153988868


In [69]:
#학습 모델 저장
torch.save(model, 'F_ori_so_model.pt')